In [1]:
%load_ext autoreload

In [ ]:
import altair as alt
import polars as pl

from altair_utils import color_value_transit_teal

In [ ]:
weekly_df = (
    pl.read_csv(
        r"Q:\Data\PeerAgencyDashboards\SFCity-EconRecovery\office_attendance-weekly-updated240614.csv"
    )
    .select(
        (pl.col("San Francisco").str.strip_chars_end("%").cast(int) / 100).alias(
            "office attendance"
        ),
        date=pl.col("Week").str.to_date("%m/%d/%Y"),
    )
    .with_columns(
        year=pl.col("date").dt.year(),
        month=pl.col("date").dt.month(),
        quarter=pl.col("date").dt.quarter(),
    )
)

In [ ]:
monthly_df = (
    weekly_df.filter(
        pl.col("date") < pl.date(2024, 11, 1)
    )  # incomplete data for Nov 2024
    .group_by("year", "month")
    .agg(pl.mean("office attendance"))
    .sort("year", "month")
    .with_columns(date=pl.date("year", "month", 1))
)
quarterly_df = (
    weekly_df.filter(
        (pl.date(2020, 4, 1) <= pl.col("date"))
        & (pl.col("date") < pl.date(2024, 10, 1))
    )
    .group_by("year", "quarter")
    .agg(pl.min("month"), pl.mean("office attendance"))
    .sort("year", "quarter")
    .with_columns(date=pl.date("year", "month", 1))
)

In [42]:
(
    alt.Chart(weekly_df)
    .mark_line()
    .encode(
        x="date",
        y=alt.Y("office attendance").axis(format="%"),
        color=color_value_transit_teal,
        tooltip=[
            alt.Tooltip("date", title="week of"),
            alt.Tooltip("office attendance", format=".0%"),
        ],
    )
)

alt.Chart(...)

In [ ]:
(
    alt.Chart(monthly_df)
    .mark_line()
    .encode(
        x="date",
        y=alt.Y("office attendance").axis(format="%"),
        color=color_value_transit_teal,
        tooltip=["year", "month", alt.Tooltip("office attendance", format=".0%")],
    )
)

alt.Chart(...)

In [ ]:
(
    alt.Chart(quarterly_df)
    .mark_line()
    .encode(
        x="date",
        y=alt.Y("office attendance").axis(format="%"),
        color=color_value_transit_teal,
        tooltip=["year", "quarter", alt.Tooltip("office attendance", format=".0%")],
    )
)

alt.Chart(...)